In [2]:
import numpy as np
import math
from scipy.optimize import linprog
from abc import ABC


In [3]:
# Test out with random dataset 

def generate_bit_data(n = 1000): 
    return np.random.randint(0,2,n)


class Mechanism(ABC): 
    pass


class LaplaceMechanism(Mechanism):
    def __init__(self, dataset, mean = 0.0, scale = 1.0):
        self.mean = mean 
        self.scale = scale
        self.dataset = dataset
        self.length = len(dataset)
    
    def query(self, indices): 
        result = np.sum(self.dataset[indices])
        noise = np.random.laplace(loc=self.mean, scale=self.scale, size=None)
        return result + noise
    
    def get_len(self):
        return self.length
    
    def get_raw_data(self):
        return self.dataset
    
    
def dinur_nissim(mech, epsilon = 0.5, gen_t = lambda n : n * (math.log2(n) ** 2)):
    n = mech.get_len()
    t = gen_t(n)
    
    def generate_query():
        # Generate a random subset of [n] by generating a list of bits and convert to indices
        q_j = np.random.randint(0,2,n)
        indices2 = q_j.nonzero()
        return (q_j, indices2) 
    
    # Make t queries to the mechanism. A_ub and b_ub are respectively the coefficient matrix 
    #   and upper bound for the linear program 
    A_ub = []
    b_ub = []
    for _ in range(int(t)):
        q, indices = generate_query()
        answer = mech.query(indices)
        A_ub.append(q)
        b_ub.append(answer + epsilon)
        A_ub.append(-1 * q)
        b_ub.append(epsilon - answer)
    print("here")
    # Rounding Phase: round the results of the LP into integers
    program_result = linprog(np.zeros(n), A_ub, b_ub, bounds=(0,1))["x"]
    round_vector = np.vectorize(round, otypes=[int])
    return round_vector(program_result)


In [5]:
data = generate_bit_data(n=10)

for s in range(10):
    mech = LaplaceMechanism(data, mean = 0, scale = (0.2 + s/100.0))
    result = dinur_nissim(mech)
    print(f"Data: {data}\n")
    print(f"Result: {result}\n")
    print(f"Errror for scale={s/100}: {np.count_nonzero(data!=result)}")
    

here
Data: [0 0 0 1 0 0 1 0 0 1]

Result: [0 0 0 1 0 0 1 0 0 1]

Errror for scale=0.0: 0
here
Data: [0 0 0 1 0 0 1 0 0 1]

Result: [0 0 0 1 0 0 1 0 0 1]

Errror for scale=0.01: 0
here
Data: [0 0 0 1 0 0 1 0 0 1]

Result: [0 0 0 1 0 0 1 0 0 1]

Errror for scale=0.02: 0
here
Data: [0 0 0 1 0 0 1 0 0 1]

Result: [0 0 0 1 0 0 1 0 0 1]

Errror for scale=0.03: 0
here
Data: [0 0 0 1 0 0 1 0 0 1]

Result: [0 0 0 1 0 0 1 0 0 1]

Errror for scale=0.04: 0
here
Data: [0 0 0 1 0 0 1 0 0 1]

Result: [0 0 0 1 0 0 1 1 0 1]

Errror for scale=0.05: 1
here
Data: [0 0 0 1 0 0 1 0 0 1]

Result: [1 1 1 1 1 1 1 1 0 1]

Errror for scale=0.06: 6
here
Data: [0 0 0 1 0 0 1 0 0 1]

Result: [0 0 0 1 0 1 1 1 0 1]

Errror for scale=0.07: 2
here
Data: [0 0 0 1 0 0 1 0 0 1]

Result: [0 1 0 1 0 0 1 0 0 1]

Errror for scale=0.08: 1
here
Data: [0 0 0 1 0 0 1 0 0 1]

Result: [0 0 0 1 0 1 1 0 0 1]

Errror for scale=0.09: 1


In [24]:
data, result


0

0